In [18]:
import os
if not os.path.exists("../input/datawid"):
    os.symlink("../input/datawid")  
    os.symlink("../input/datawid") 
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex3 import *
print("Setup Complete")

Setup Complete


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = pd.read_csv('../input/datawid/insurance.csv', index_col='ID') 
X_test = pd.read_csv('../input/datawid/insurance.csv', index_col='ID')

#Predicting the expenses so do not keep in.
X.dropna(axis=0, subset=['expenses'], inplace=True)
y = X.expenses
X.drop(['expenses'], axis=1, inplace=True)

#Getting rid of columns with null values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)
X_train.head()

,age,sex,bmi,children,smoker,region
ID,,,,,,
622,37,male,34.1,4,yes,southwest
195,18,male,34.4,0,no,southeast
241,23,female,36.7,2,yes,northeast
1169,32,male,35.2,2,no,southwest
1193,58,female,32.4,1,no,northeast


In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [22]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['sex', 'smoker', 'region']

Categorical columns that will be dropped from the dataset: []


In [27]:
from sklearn.preprocessing import LabelEncoder

# Drop cats that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Drop cats that will encode
label_encoder = LabelEncoder()
for col in set(good_label_cols):
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])
    
print("MAE from Categorical Encoding:") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))



MAE from Categorical Encoding:
2710.5810980223873


In [17]:
# Returns the number of unique values w/in a cat. (e.g. "Smoker?" Yes or No,aka "cardinality")
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

sorted(d.items(), key=lambda x: x[1])

# One Hot Coding not necessary because no cats have cardinality >10.

[('sex', 2), ('smoker', 2), ('region', 4)]